<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0623BERT_SNOW_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 「やさしい日本語」コーパスを使って BERT を訓練する実習

- 浅川伸一
- filename: 2022_0623BERT_SNOW_trainign.ipynb

In [ ]:
%config InlineBackend.figure_format = 'retina'
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

from termcolor import colored
import platform
HOSTNAME = platform.node().split('.')[0]

import os
HOME = os.environ['HOME']

try:
    import ipynbname
except ImportError:
    !pip install ipynbname > /dev/null
import ipynbname
FILEPATH = str(ipynbname.path()).replace(HOME+'/','')

import pwd
USER=pwd.getpwuid(os.geteuid())[0]

from datetime import date
TODAY=date.today()

import torch
TORCH_VERSION = torch.__version__

color = 'green'
print('日付:',colored(f'{TODAY}', color=color, attrs=['bold']))
print('HOSTNAME:',colored(f'{HOSTNAME}', color=color, attrs=['bold']))
print('ユーザ名:',colored(f'{USER}', color=color, attrs=['bold']))
print('HOME:',colored(f'{HOME}', color=color,attrs=['bold']))
print('ファイル名:',colored(f'{FILEPATH}', color=color, attrs=['bold']))
print('torch.__version__:',colored(f'{TORCH_VERSION}', color=color, attrs=['bold']))

In [ ]:
if isColab:
    !pip install jaconv
    !pip install transformers fugashi ipadic
    
import os
import pandas as pd
import requests
from termcolor import colored
import jaconv

# やさしい日本語をダウンロード
SNOWs={'T15': {'url':"https://filedn.com/lit4DCIlHwxfS1gj9zcYuDJ/SNOW/T15-2020.1.7.xlsx"},
       'T23': {'url':"https://filedn.com/lit4DCIlHwxfS1gj9zcYuDJ/SNOW/T23-2020.1.7.xlsx"},
      }
print('エクセルファイル読込', end='...')
for corpus in SNOWs:
    url = SNOWs[corpus]['url']
    excel_fname = corpus + '-2020.1.7.xlsx'
    if not os.path.exists(excel_fname):  # ファイルが存在しない場合，ダウンロードする
        print(f'url:{url}')
        r = requests.get(url)
        with open(excel_fname, 'wb') as f:
            total_length = int(r.headers.get('content-length'))
            print(f'{excel_fname} をダウンロード中 {total_length} バイト')
            f.write(r.content)

    SNOWs[corpus]['df'] = pd.read_excel(excel_fname)
    SNOWs[corpus]['df'] = SNOWs[corpus]['df'].rename(columns={'#日本語(原文)': 'ja', 
                                                              '#やさしい日本語':'easy_ja',
                                                              '#英語(原文)':'en'})
# 2 つのデータをあわせる    
_snow = SNOWs['T15']['df']['easy_ja'].tolist() + SNOWs['T23']['df']['easy_ja'].tolist()
snow = [jaconv.normalize(line, 'NFKC') for line in _snow] # 正規化

BERT は マスク化言語モデル (**MLM**) と次文予測 (**NSP**) という 2 つの独自の学習アプローチがある。

モデルを微調整 fine-tuning する必要がある

# 1. マスク化言語モデル MLM

MLM は BERT に文を与え，BERT 内部の重みを最適化して，相手側に同じ文を出力することで構成されています。
すなわち，文を入力し，BERT が同じ文を出力するように要求します。

しかし，実際に BERT にその入力文を与える前に，いくつかのトークンを必要とします。

<center>
<img src="https://miro.medium.com/max/1400/1*phTLnQ8itb3ZX5_h9BWjWw.png" width="600px"><br/>
この画像では，トークン を BERT に渡す前に，リンカーン・トークンを [MASK] に置き換えてマスクしています。
</center>

つまり，実際には不完全な文章を入力して，BERT に完成させるように依頼しているのです。

## 1.1 間隙を埋める

これはどのような効果があるのでしょうか？
それは、多くの人が学校で与えられた問題のようなものです。つまり、文章が与えられたら、その穴を埋めなければなりません。

`秋 に は ， ___ が 木 から 落ちる 。`

答えがわかりますか？
ほとんど、あなたは分かっているでしょう。
それは，文脈を考慮したからです。


「落ちる」 と 「木」 という単語が出てきましたが，足りない単語は木から落ちるものだということがわかります。

どんぐり，枝，葉など，木から落ちるものはたくさんありますが，秋という別の条件があるので，秋に木から落ちる可能性が最も高いのは葉だということで，検索対象が絞られます。

人間として，私たちは一般的な世界の知識と言語的な理解を組み合わせて，その結論を導き出します。
BERT の場合，この推測は，たくさんの本を読み，言語パターンを非常によく学んでいることから得られます。

BERT は，秋，木，葉が何であるかを知らないかもしれませんが，言語パターンとこれらの単語の文脈から，答えが葉である可能性が最も高いことを知っています。

この処理の結果，BERT にとっては，使用されている言語のスタイルの理解度が向上します。


## 1.2 実際の処理

MLM が何をしているかは理解できましたが，実際にはどのように機能するのでしょうか？
コード上で必要となる論理的なステップは何でしょうか？

1.  テキストをトークン化します。
通常の変換機と同じように，まずテキストのトークン化を行います。
トークン化からは 3 つの異なるテンソルが得られます。

* input_ids
* token_type_ids
* attention_mask

MLM には token_type_ids は必要ないし，この例では attention_mask はそれほど重要ではありません。

我々にとっては input_ids テンソルが最も重要です。
ここには，トークン化されたテキスト表現があり，これを今後修正していくことになるでしょう。

2. `labels tensor` を作成する。
ここではモデルを訓練しているので，損失を計算して最適化するためのラベルテンソルが必要です。
`labels tensor` は単純に `input_ids` なので，これをコピーするだけです。


3. `input_ids` のトークンをマスクする。
label 用の `input_ids` のコピーを作成したので，先にトークンのランダムな選択をマスクすることができます。


BERT 論文では，モデルの事前訓練中に，いくつかの追加ルールを用いて，各トークンを 15％ の確率でマスク化していますが，ここではこれを簡略化して，各単語を 15％ の確率でマスク化することにします。


4. 損失を計算します。
`input_ids` と `labels` のテンソルを BERT モデルで処理し，両者の間の損失を計算します。
この損失を用いて，BERT による必要な勾配変化を計算し，モデルの重みを最適化します。

<center>
<img src="https://miro.medium.com/max/1400/1*0KvOrY6rY055m9oq36HRkg.png" width="600px"><br/>
<div style="text-align:left; width:66%; background-color:cornsilk">

512 個のトークンはすべて，モデルの語彙サイズに等しいベクトル長を持ちます。
最終的な出力埋め込みベクトルであるロジット(確率比) を生成します。
予測されたトークン ID は，lソフトマックスと argmax 変換を用いて，このロジットから抽出されます。
</div>    
</center>    

損失は，各出力「トークン」の出力確率分布と，真のワンホット符号化ラベルとの差として計算されます。


# 2. マスク化言語モデル MLM のコード

MLM をコードで実証するにはどうすればよいでしょうか？

HuggingFace のトランスフォーマーと PyTorch そして bert-base-uncased モデルを使用します。
では，まず全てをインポートして初期化しましょう。

In [ ]:
import torch
import transformers
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM

# BERT 訓練済モデルをダウロード
# model_name_ja = 'cl-tohoku/bert-base-japanese-whole-word-masking'
model_name_ja = 'cl-tohoku/bert-base-japanese'  # 東北大学乾研による 日本語 BERT 実装
# see https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2
# model_name_ja = 'sonoisa/sentence-bert-base-ja-mean-tokens-v2'  # 東北大学乾研による 日本語 BERT 実装

tknz = BertJapaneseTokenizer.from_pretrained(model_name_ja)
bert_model = BertForMaskedLM.from_pretrained(model_name_ja, return_dict = True)


In [ ]:
max_length = 0
for line in snow:
    max_length = len(line) if len(line) > max_length else max_length
print(f'やさしい日本語における一文の最長文字数:{max_length}')
inputs = tknz(snow, return_tensors='pt', max_length=max_length+1, truncation=True, padding='max_length')
# トークンの分割数が文字数以上になることはないので `max_length=max_length+1`とした
print(inputs.input_ids.shape)

In [ ]:
#type(tknz.vocab)  # collections.OrderedDict
#len(tknz.vocab)    # 3200
print(list(tknz.vocab.keys())[-30:])
print(tknz.tokenize(snow[0]))
print(tknz(snow[0])['input_ids'])

## 文章のトークン化

In [ ]:
print(snow[:3])
inputs = tknz(snow[:3], return_tensors='pt', padding=True)
print(inputs.keys())
print(inputs)

## ラベル作成

`input_ids` テンソルを新しい labels テンソルにクローンして，
結果を `inputs` 変数に格納します。


In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
print(inputs)

## マスクの作成

BERT の原著論文では，マスク化率が 15% なので，この割合でランダムなマスクを作成します。

トークンを 15% の確率でマスク化するためには `torch.rand` と各値 $<0.15$ という条件を用い，
マスク配列 `mask_arr` を生成。

In [ ]:
# input_idsと同じ次元の浮動小数点数のランダムな配列を作る
rand = torch.rand(inputs.input_ids.shape)

# 乱数配列が 0.15 より小さい場合は true を設定
mask_arr = rand < 0.15
print(mask_arr)

[MASK] トークンを配置する場所を選ぶために mask_arr を使いますが，[CLS] トークンや [SEP] トークンなどの他の特殊トークン (それぞれ `tknz.cls_token_id` と `tknz.sep_token_id` で取得可能) の上に MASK トークンを配置したくはありません。

そこで，さらに条件を追加する必要があります。
トークン ID `tknz.cls_token_id` または `tknz.sep_token_id` を含む位置のチェック。

In [ ]:
print(tknz.vocab['[MASK]'], '=', tknz.mask_token_id)
print(tknz.vocab['[CLS]'], '=', tknz.cls_token_id)
print(tknz.vocab['[SEP]'], '=', tknz.sep_token_id)

In [ ]:
print(tknz.special_tokens_map)
print([(value,tknz.vocab[value]) for token, value in tknz.special_tokens_map.items()])

In [ ]:
print((inputs.input_ids != tknz.cls_token_id) * (inputs.input_ids != tknz.sep_token_id))

In [ ]:
mask_arr = (rand < 0.15) * (inputs.input_ids != tknz.cls_token_id) * (inputs.input_ids != tknz.sep_token_id)
print(mask_arr)

In [ ]:
# mask_arr から selection を作成
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()
print(selection)

In [ ]:
# selection インデックスを inputs.input_ids へ適用し MASK トークンを追加 
inputs.input_ids[0, selection] = tknz.vocab[tknz.mask_token]
print(inputs)

これで，上の `input_ids` テンソルで  103 で表された MASK トークンを見ることができます。

## 損失の計算

最後のステップは，一般的なモデルの訓練処理と変わりません。
`input_ids` テンソルと `labels` テンソルが `input dictionary` に入っているので，これをモデルに渡してモデルの損失を返すことができます。

In [ ]:
outputs = bert_model(**inputs)

In [ ]:
# outputs には loss と logits とがあります。
print(outputs.keys())

In [ ]:
print(outputs.loss)

# 学習の実装


In [ ]:
%%time
inputs = tknz(snow, return_tensors='pt', max_length=100, truncation=True, padding='max_length')
print(inputs)

In [ ]:
print(inputs['input_ids'].size())
print(inputs.input_ids.size())
print(type(inputs.input_ids.detach()))
print(type(inputs.input_ids.detach().numpy()))
print(inputs.input_ids.detach().numpy().shape)
print(inputs.input_ids.detach().numpy()[:2])
print(type(snow), len(snow))

次に `input_id` をクローンして，ラベルテンソルを作成します。


In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
print(inputs.keys())


次に，マスクのコードですが，マスクに PAD トークンを含めてはいけない (CLS や SEP では以前のとおりにあつかう)。

In [ ]:
print(tknz.pad_token_id)
print(tknz.mask_token_id)
print(tknz.sep_token_id)

In [ ]:
# input_ids テンソルと同じ次元の浮動小数点数のランダムな配列を作成 
rand = torch.rand(inputs.input_ids.shape)

# mask 配列の作成 
mask_arr = (rand < 0.15) * (inputs.input_ids != tknz.cls_token_id) * \
           (inputs.input_ids != tknz.sep_token_id) * (inputs.input_ids != tknz.pad_token_id)
print(mask_arr)

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )
print(selection[:5])


次に，これらのインデックスを `input_ids` の各行に適用し，これらのインデックスの値をそれぞれ `tknz.mask_token_id` として割り当てます。


In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = tknz.mask_token_id

print(tknz.mask_token_id)
print(inputs.input_ids[:3])


`mask_arr` テンソルの `True` 値と同じ位置に `tknz.mask_token_id` が割り当てられている

これで入力テンソルの準備が整い，学習時にモデルに入力するための設定を始めることができる。

学習時には PyTorch の `DataLoader` を使ってデータを読み込みます。
これを使うには，データを PyTorch の `Dataset` オブジェクトにフォーマットする必要がある


In [ ]:
class snowDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings.input_ids)
    
dataset = snowDataset(inputs) 

`dataloader` を初期化する。
`dataloader` は，訓練時にモデルにデータを読み込むために使用。


In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

これで，反復訓練に入る準備が整った。
反復学習を始める前に，3 つのことを設定する必要がある。

1. モデルを GPU/CPU (GPU が利用可能あれば) に移動させる
2. モデルの訓練モードを有効にする
3. 重み付けされた重み崩壊付き最適化 `AdamW` を初期化する


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

bert_model.to(device) # モデルを選択したデバイスに移動
bert_model.train();   # 訓練モードに設定

In [ ]:
from torch.optim import AdamW

optim = AdamW(bert_model.parameters(), lr=5e-5) #最適化関数を初期化

これでようやくセットアップが完了し，訓練を開始することができる。
ここでは PyTorch の典型的な訓練ループを導入する。


In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 2
for epoch in range(epochs):

    loop = tqdm(loader, leave=True)
    for batch in loop:
        
        optim.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()
        
        loop.set_description(f'エポック {epoch}')
        #loop.set_postfix(f'損失 {loss.item()}')
        loop.set_postfix(loss=loss.item())        